In [1]:
# -*- coding: utf-8 -*-

機械学習で東京23区のお買い得物件を探してみる

# ライブラリーインポート

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import tqdm
import csv
import collections
from collections import Counter

# データ取得の検討

BeautifulSoupでSuumoよりスクレイピング。

## 足立区のurl取得

## データ（属性）取得

### 情報の取得方法（検討用）

=> これで実施していく。

## データ取得実施

In [42]:
# データ（属性）を格納する変数を作成
titles = [] #建物名（違う場合もあるが念のため）
ltypes = [] #レンタルタイプ（賃貸アパート、賃貸マンション、など）
addresses = [] #住所
location0s = [] #立地1つ目（最寄駅/徒歩~分）
location1s = [] #立地2つ目（最寄駅/徒歩~分）
location2s = [] #立地3つ目（最寄駅/徒歩~分）
ages = [] #築年数
heights = [] #建物高さ
floors = [] #階
rents = [] #賃料
admins = [] #管理費
others = [] #敷/礼/保証/敷引,償却
floor_plans = [] #間取り
areas = [] #専有面積

In [44]:
rooms = 0 # 物件数
i = 0 # ページ数
for url in urls:
    i += 1 #ページ数カウント
    # 物件情報が入っている箇所を取得
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c)
    summary = soup.find("div",{"id":"js-bukkenList"})

    # 物件（建物）ごとの情報を取得
    cassetteitems = summary.find_all("div",{"class":"cassetteitem"})

    rooms += len(cassetteitems) #物件数カウント
    print("{}/{}\t物件（建物）数：{}".format(str(i), str(len(urls)), len(cassetteitems)))
    # 各物件で情報を取得
    for cassetteitem in cassetteitems:
        # 部屋数を取得
        tbodies = cassetteitem.find_all("tbody")

        # 建物情報を取得：レンタルタイプ、建物名、住所、立地（3つまで）、築年数、建物高さ
        # - レンタルタイプ
        ltype = cassetteitem.find("span",{"class":"ui-pct ui-pct--util1"}).find(text=True)
        # - 建物名
        title = cassetteitem.find("div",{"class":"cassetteitem_content-title"}).find(text=True)
        # - 住所
        address = cassetteitem.find("li",{"class":"cassetteitem_detail-col1"}).find(text=True)
        # - 立地（3つまで）
        locations = cassetteitem.find_all("div",{"class":"cassetteitem_detail-text"})
        l = len(locations)
        if l == 1:
            location0 = str(locations[0].find(text=True))
        if l == 2:
            location0 = str(locations[0].find(text=True))
            location1 = str(locations[1].find(text=True))
        else:
            location0 = str(locations[0].find(text=True))
            location1 = str(locations[1].find(text=True))
            location2 = str(locations[2].find(text=True))
        # 築年数、建物高さ
        ah = cassetteitem.find("li",{"class":"cassetteitem_detail-col3"})
        ah = ah.find_all("div")
        age = str(ah[0].find(text=True))
        height = str(ah[1].find(text=True))

        # 部屋情報を取得：階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積
        for tbody in tbodies:
            cassetteitem_others = tbody.find_all("td")
            # 階
            floor = str(cassetteitem_others[2].find(text=True))
            # 賃料
            rent = str(cassetteitem_others[3].find("span",{"class":"cassetteitem_other-emphasis ui-text--bold"}).find(text=True))
            # 管理費
            admin = str(cassetteitem_others[4].find(text=True))
            # 敷/礼/保証/敷引,償却
            other = str(cassetteitem_others[5].find(text=True))
            # 間取り
            floor_plan = str(cassetteitem_others[6].find(text=True))
            # 専有面積
            area = str(cassetteitem_others[7].find(text=True))

            # 部屋ごとにデータ（属性）を格納
            titles.append(title) #建物名
            ltypes.append(ltype) #レンタルタイプ
            addresses.append(address) #住所
            location0s.append(location0) #立地1つ目
            location1s.append(location1) #立地2つ目
            location2s.append(location2) #立地3つ目
            ages.append(age) #築年数
            heights.append(height) #建物高さ
            floors.append(floor) #階
            rents.append(rent) #賃料
            admins.append(admin) #管理費
            others.append(other) #敷/礼/保証/敷引,償却
            floor_plans.append(floor_plan) #間取り
            areas.append(area) #専有面積            
            
    # プログラムを最低1秒間停止する（スクレイピングマナー）
    time.sleep(3)

print("ページ数：{}\t部屋数：{}".format(str(i), str(rooms)))

1/219


/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


物件（建物）数： 30
2/219
物件（建物）数： 30
3/219
物件（建物）数： 30
4/219
物件（建物）数： 30
5/219
物件（建物）数： 30
6/219
物件（建物）数： 30
7/219
物件（建物）数： 30
8/219
物件（建物）数： 30
9/219
物件（建物）数： 30
10/219
物件（建物）数： 30
11/219
物件（建物）数： 30
12/219
物件（建物）数： 30
13/219
物件（建物）数： 30
14/219
物件（建物）数： 30
15/219
物件（建物）数： 30
16/219
物件（建物）数： 30
17/219
物件（建物）数： 30
18/219
物件（建物）数： 30
19/219
物件（建物）数： 30
20/219
物件（建物）数： 30
21/219
物件（建物）数： 30
22/219
物件（建物）数： 30
23/219
物件（建物）数： 30
24/219
物件（建物）数： 30
25/219
物件（建物）数： 30
26/219
物件（建物）数： 30
27/219
物件（建物）数： 30
28/219
物件（建物）数： 30
29/219
物件（建物）数： 30
30/219
物件（建物）数： 30
31/219
物件（建物）数： 30
32/219
物件（建物）数： 30
33/219
物件（建物）数： 30
34/219
物件（建物）数： 30
35/219
物件（建物）数： 30
36/219
物件（建物）数： 30
37/219
物件（建物）数： 30
38/219
物件（建物）数： 30
39/219
物件（建物）数： 30
40/219
物件（建物）数： 30
41/219
物件（建物）数： 30
42/219
物件（建物）数： 30
43/219
物件（建物）数： 30
44/219
物件（建物）数： 30
45/219
物件（建物）数： 30
46/219
物件（建物）数： 30
47/219
物件（建物）数： 30
48/219
物件（建物）数： 30
49/219
物件（建物）数： 30
50/219
物件（建物）数： 30
51/219
物件（建物）数： 30
52/219
物件（建物）数： 30
53/219
物件（建物）数： 30
54/219
物

=> 219/222:4 で、サイトを調べたところ、219ページまでしかなかった。<br>
=> urls取得日からデータ取得日が4日後のため情報が更新されていたことが原因のため、エラーは無視して作業を進める。

In [45]:
print("ページ数：", str(i))
print("物件数：", str(rooms))
print("部屋数：", str(len(ltypes)))

ページ数： 219
物件数： 6553
部屋数： 17174


## データフレーム作成・csv保存

In [46]:
# データ格納した変数をSeries化
titles = pd.Series(titles) #建物名
ltypes = pd.Series(ltypes) #レンタルタイプ
addresses = pd.Series(addresses) #住所
location0s = pd.Series(location0s) #立地1つ目
location1s = pd.Series(location1s) #立地2つ目
location2s = pd.Series(location2s) #立地3つ目
ages = pd.Series(ages) #築年数
heights = pd.Series(heights) #建物高さ
floors = pd.Series(floors) #階
rents = pd.Series(rents) #賃料
admins = pd.Series(admins) #管理費
others = pd.Series(others) #敷/礼/保証/敷引,償却
floor_plans = pd.Series(floor_plans) #間取り
areas = pd.Series(areas) #専有面積

In [47]:
# 各SeriesをDataFrame化
df = pd.concat([titles, ltypes, addresses, location0s, location1s, location2s, ages, heights, floors, rents, admins, others, floor_plans, areas], axis = 1)

# 列名を追加
df.columns = ["建物名", "レンタルタイプ", "住所", "立地1", "立地2", "立地3", "築年数", "建物高さ", "階", "賃料", "管理費", "敷/礼/保証/敷引,償却", "間取り", "専有面積"]

In [48]:
# .csvファイル出力
df.to_csv("../output/suumo_adachi_20171115.csv", sep = ",", encoding = "utf-8")

# データ前処理の検討

## csvからdf作成

In [2]:
# csv読み込み
df = pd.read_csv("../output/info/info_adachi_20171119.csv", encoding="utf-8")
df

,Unnamed: 0,建物名,レンタルタイプ,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積
0,0,東京メトロ千代田線 綾瀬駅 2階建 築33年,賃貸アパート,東京都足立区西綾瀬４,東京メトロ千代田線/綾瀬駅 歩11分,つくばエクスプレス/青井駅 歩12分,東武伊勢崎線/小菅駅 歩15分,築33年,2階建,2階,2.5万円,1000円,2.5万円/2.5万円/-/-,2K,24m
1,1,小泉荘,賃貸アパート,東京都足立区小台２,日暮里・舎人ライナー/足立小台駅 歩17分,ＪＲ高崎線/尾久駅 歩27分,日暮里・舎人ライナー/熊野前駅 歩21分,築48年,2階建,1階,2.6万円,-,2.6万円/2.6万円/-/-,ワンルーム,11.57m
2,2,東京メトロ南北線 王子神谷駅 2階建 築43年,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩15分,None,None,築43年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,1K,12m
3,3,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,2階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
4,4,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
5,5,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
6,6,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
7,7,柳荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,京成本線/京成関屋駅 歩1分,東武伊勢崎線/牛田駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
8,8,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,1階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
9,9,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m


In [3]:
# 不要カラム削除
df = df.ix[:, 1:]
#df.drop("Unnamed: 0", axis=1, inplace=True) でも可
df

/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


,建物名,レンタルタイプ,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積
0,東京メトロ千代田線 綾瀬駅 2階建 築33年,賃貸アパート,東京都足立区西綾瀬４,東京メトロ千代田線/綾瀬駅 歩11分,つくばエクスプレス/青井駅 歩12分,東武伊勢崎線/小菅駅 歩15分,築33年,2階建,2階,2.5万円,1000円,2.5万円/2.5万円/-/-,2K,24m
1,小泉荘,賃貸アパート,東京都足立区小台２,日暮里・舎人ライナー/足立小台駅 歩17分,ＪＲ高崎線/尾久駅 歩27分,日暮里・舎人ライナー/熊野前駅 歩21分,築48年,2階建,1階,2.6万円,-,2.6万円/2.6万円/-/-,ワンルーム,11.57m
2,東京メトロ南北線 王子神谷駅 2階建 築43年,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩15分,None,None,築43年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,1K,12m
3,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,2階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
4,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
5,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
6,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
7,柳荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,京成本線/京成関屋駅 歩1分,東武伊勢崎線/牛田駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
8,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,1階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
9,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m


## 賃料がNAの行を削除

In [4]:
# サイトを参考に、賃料がNAの行を削除（足立区データにはなかったが必要な処理）
df = df.dropna(subset=["賃料"])
df

,建物名,レンタルタイプ,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積
0,東京メトロ千代田線 綾瀬駅 2階建 築33年,賃貸アパート,東京都足立区西綾瀬４,東京メトロ千代田線/綾瀬駅 歩11分,つくばエクスプレス/青井駅 歩12分,東武伊勢崎線/小菅駅 歩15分,築33年,2階建,2階,2.5万円,1000円,2.5万円/2.5万円/-/-,2K,24m
1,小泉荘,賃貸アパート,東京都足立区小台２,日暮里・舎人ライナー/足立小台駅 歩17分,ＪＲ高崎線/尾久駅 歩27分,日暮里・舎人ライナー/熊野前駅 歩21分,築48年,2階建,1階,2.6万円,-,2.6万円/2.6万円/-/-,ワンルーム,11.57m
2,東京メトロ南北線 王子神谷駅 2階建 築43年,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩15分,None,None,築43年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,1K,12m
3,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,2階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
4,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
5,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
6,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
7,柳荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,京成本線/京成関屋駅 歩1分,東武伊勢崎線/牛田駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
8,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,1階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
9,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m


## 列を分割

### 住所

In [5]:
# 住所を都道府県、市区町村、町域以下、に分割（東京都/〇〇区/それ以下）
df_address1 = df["住所"].str.split("都", expand=True)
df_address1.columns = ["都道府県", "それ以下"]
df_address2 = df_address1["それ以下"].str.split("区", expand=True)
df_address2.columns = ["市区町村", "町域以下"]
df_address = pd.concat([df_address1["都道府県"], df_address2], axis=1)
# データ確認
df_address

,都道府県,市区町村,町域以下
0,東京,足立,西綾瀬４
1,東京,足立,小台２
2,東京,足立,新田１
3,東京,足立,新田１
4,東京,足立,新田１
5,東京,足立,梅島２
6,東京,足立,梅島２
7,東京,足立,千住曙町
8,東京,足立,千住曙町
9,東京,足立,千住曙町


### 立地

In [6]:
np.unique(df["立地1"])

array(['つくばエクスプレス/八潮駅 バス10分 (バス停)佐野センター 歩5分', 'つくばエクスプレス/八潮駅 歩13分',
       'つくばエクスプレス/八潮駅 歩15分', 'つくばエクスプレス/八潮駅 歩16分', 'つくばエクスプレス/八潮駅 歩17分',
       'つくばエクスプレス/八潮駅 歩18分', 'つくばエクスプレス/八潮駅 歩19分', 'つくばエクスプレス/八潮駅 歩20分',
       'つくばエクスプレス/八潮駅 歩22分', 'つくばエクスプレス/八潮駅 歩23分', 'つくばエクスプレス/八潮駅 歩24分',
       'つくばエクスプレス/八潮駅 歩25分', 'つくばエクスプレス/八潮駅 歩26分', 'つくばエクスプレス/八潮駅 歩27分',
       'つくばエクスプレス/八潮駅 歩28分', 'つくばエクスプレス/八潮駅 歩29分', 'つくばエクスプレス/八潮駅 歩31分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)樫の木公園 歩3分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)樫の木公園前 歩1分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)第五都営住宅 歩1分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)花畑交番前 歩3分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)花畑第一小学校入り口 歩1分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)花畑第一小学校入口 歩1分',
       'つくばエクスプレス/六町駅 バス10分 (バス停)花畑記念公園入口 歩3分',
       'つくばエクスプレス/六町駅 バス12分 (バス停)樫 歩4分',
       'つくばエクスプレス/六町駅 バス12分 (バス停)樫の木 歩3分',
       'つくばエクスプレス/六町駅 バス12分 (バス停)特別支援学校前 歩3分',
       'つくばエクスプレス/六町駅 バス12分 (バス停)第４都営住宅 歩3分',
       'つくばエクスプレス/六町駅 バス12分 (バス停)花畑4丁目 歩2分',
       'つくばエクスプレ

- 「◇路線/△駅 歩◯分」「◇路線/△駅 バス□分 歩◯分」の２種類がある。
- => ["路線", "駅", "バス", "徒歩"]:[◇,△,□,◯]　とする。（※バスがない場合は□=0）

#### ↓星バス、なども考慮し修正が必要

In [272]:
# 1〜3それぞれで、路線、駅、徒歩に分割
'''
□路線/〇駅 歩△分 => ["路線", "駅", "徒歩"]:[□, 〇, △]
'''
df_location0_1 = df["立地1"].str.split("/", expand=True)
df_location0_1.columns = ["路線", "その他"]
df_location0_2 = df_location0_1["その他"].str.split(" 歩", expand=True)
df_location0_2.columns = ["駅", "徒歩単位"]
df_location0_3 = df_location0_2["徒歩単位"].str.split("分", expand=True)
df_location0_3.columns = ["徒歩", "単位"]
df_location0 = pd.concat([df_location0_1["路線"], df_location0_2["駅"], df_location0_3["徒歩"]], axis=1)
#df_location0
df_location1_1 = df["立地2"].str.split("/", expand=True)
df_location1_1.columns = ["路線", "その他"]
df_location1_2 = df_location1_1["その他"].str.split(" 歩", expand=True)
df_location1_2.columns = ["駅", "徒歩単位"]
df_location1_3 = df_location1_2["徒歩単位"].str.split("分", expand=True)
df_location1_3.columns = ["徒歩", "単位"]
df_location1 = pd.concat([df_location1_1["路線"], df_location1_2["駅"], df_location1_3["徒歩"]], axis=1)
#df_location1
df_location2_1 = df["立地3"].str.split("/", expand=True)
df_location2_1.columns = ["路線", "その他"]
df_location2_2 = df_location2_1["その他"].str.split(" 歩", expand=True)
df_location2_2.columns = ["駅", "徒歩単位"]
df_location2_3 = df_location2_2["徒歩単位"].str.split("分", expand=True)
df_location2_3.columns = ["徒歩", "単位"]
df_location2 = pd.concat([df_location2_1["路線"], df_location2_2["駅"], df_location2_3["徒歩"]], axis=1)
#df_location2

In [273]:
df

,建物名,レンタルタイプ,住所,立地1,立地2,立地3,築年数,建物高さ,階,賃料,管理費,"敷/礼/保証/敷引,償却",間取り,専有面積
0,東京メトロ千代田線 綾瀬駅 2階建 築33年,賃貸アパート,東京都足立区西綾瀬４,東京メトロ千代田線/綾瀬駅 歩11分,つくばエクスプレス/青井駅 歩12分,東武伊勢崎線/小菅駅 歩15分,築33年,2階建,2階,2.5万円,1000円,2.5万円/2.5万円/-/-,2K,24m
1,小泉荘,賃貸アパート,東京都足立区小台２,日暮里・舎人ライナー/足立小台駅 歩17分,ＪＲ高崎線/尾久駅 歩27分,日暮里・舎人ライナー/熊野前駅 歩21分,築48年,2階建,1階,2.6万円,-,2.6万円/2.6万円/-/-,ワンルーム,11.57m
2,東京メトロ南北線 王子神谷駅 2階建 築43年,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩15分,None,None,築43年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,1K,12m
3,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,2階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
4,文月荘,賃貸アパート,東京都足立区新田１,東京メトロ南北線/王子神谷駅 歩17分,None,None,築53年,2階建,1階,2.5万円,1500円,2.5万円/2.5万円/-/-,ワンルーム,10.01m
5,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
6,豊和荘,賃貸アパート,東京都足立区梅島２,東武伊勢崎線/梅島駅 歩9分,東武伊勢崎線/西新井駅 歩13分,東武大師線/大師前駅 歩23分,築58年,2階建,1階,2.8万円,-,2.8万円/-/-/-,ワンルーム,10m
7,柳荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,京成本線/京成関屋駅 歩1分,東武伊勢崎線/牛田駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
8,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,1階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m
9,柳川荘,賃貸アパート,東京都足立区千住曙町,東京メトロ千代田線/北千住駅 歩15分,東武伊勢崎線/牛田駅 歩1分,京成本線/京成関屋駅 歩1分,築47年,2階建,2階,2.8万円,-,2.8万円/2.8万円/-/-,ワンルーム,10m


In [274]:
df_location0

,路線,駅,徒歩
0,東京メトロ千代田線,綾瀬駅,11
1,日暮里・舎人ライナー,足立小台駅,17
2,東京メトロ南北線,王子神谷駅,15
3,東京メトロ南北線,王子神谷駅,17
4,東京メトロ南北線,王子神谷駅,17
5,東武伊勢崎線,梅島駅,9
6,東武伊勢崎線,梅島駅,9
7,東京メトロ千代田線,北千住駅,15
8,東京メトロ千代田線,北千住駅,15
9,東京メトロ千代田線,北千住駅,15


In [275]:
df_location1

,路線,駅,徒歩
0,つくばエクスプレス,青井駅,12
1,ＪＲ高崎線,尾久駅,27
2,None,None,None
3,None,None,None
4,None,None,None
5,東武伊勢崎線,西新井駅,13
6,東武伊勢崎線,西新井駅,13
7,京成本線,京成関屋駅,1
8,東武伊勢崎線,牛田駅,1
9,東武伊勢崎線,牛田駅,1


In [276]:
df_location2

,路線,駅,徒歩
0,東武伊勢崎線,小菅駅,15
1,日暮里・舎人ライナー,熊野前駅,21
2,None,None,None
3,None,None,None
4,None,None,None
5,東武大師線,大師前駅,23
6,東武大師線,大師前駅,23
7,東武伊勢崎線,牛田駅,1
8,京成本線,京成関屋駅,1
9,京成本線,京成関屋駅,1


### 築年数

=> 「新築」を「築0年」と変換して数値を取得する。

In [13]:
# 築年数から数値のみ取得
df_age = df["築年数"].str.replace("新築", "築0年")
df_age = df_age.str.extract("([0-9]+)").astype(int)
df_age.columns = ["築年数"]
# データ確認
Counter(df_age)

/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


Counter({0: 1845,
         1: 134,
         2: 427,
         3: 333,
         4: 207,
         5: 250,
         6: 172,
         7: 236,
         8: 221,
         9: 248,
         10: 259,
         11: 330,
         12: 168,
         13: 224,
         14: 175,
         15: 141,
         16: 151,
         17: 115,
         18: 124,
         19: 93,
         20: 82,
         21: 164,
         22: 142,
         23: 233,
         24: 286,
         25: 294,
         26: 414,
         27: 443,
         28: 433,
         29: 486,
         30: 465,
         31: 509,
         32: 240,
         33: 229,
         34: 101,
         35: 143,
         36: 114,
         37: 126,
         38: 138,
         39: 114,
         40: 122,
         41: 121,
         42: 85,
         43: 110,
         44: 73,
         45: 70,
         46: 39,
         47: 32,
         48: 56,
         49: 21,
         50: 9,
         51: 48,
         52: 33,
         53: 47,
         54: 6,
         55: 15,
         56: 4,
  

### 建物高さ

- ほとんどの建物に地下がないことがわかる。
- => 部屋の階数変換として、地下はマイナス値とするため、項目「地上」と「地下」にし、地下はマイナス値とする。
- 「平屋」は1階建とする。

In [14]:
# 平屋を1階建に変換
df_height1 = df["建物高さ"].str.replace("平屋", "1階建")
# 地下と地上に分割
df_height1 = pd.DataFrame(["地上" + x if "地上" not in x else x for x in df_height1], columns=["建物高さ"])
df_height2 = df_height1["建物高さ"].str.split("地上", expand=True)
df_height2.columns = ["地下", "地上"]
df_height2["地下"] = ["地下0" if x == "" else x for x in df_height2["地下"]]
# 地下と地上の数値のみ取得
df_height2["地下"] = df_height2["地下"].str.extract("([0-9]+)").astype(int)
df_height2["地上"] = df_height2["地上"].str.extract("([0-9]+)").astype(int)
# 地下はマイナス値にする
df_height2["地下"] = df_height2["地下"] * -1
# データ確認
print(Counter(df_height2["地下"]))
print(Counter(df_height2["地上"]))
df_height2

Counter({0: 11858, -1: 55, -2: 13, -5: 1, -4: 1, -3: 1})
Counter({2: 4045, 3: 3848, 4: 1043, 5: 869, 6: 485, 10: 410, 7: 339, 9: 286, 8: 224, 14: 177, 11: 94, 12: 46, 15: 19, 1: 18, 13: 16, 16: 5, 18: 2, 19: 2, 42: 1})


/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:9: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.


,地下,地上
0,0,2
1,0,2
2,0,2
3,0,2
4,0,2
5,0,2
6,0,2
7,0,2
8,0,2
9,0,2


### 階

- 地下の部屋は1件しかない。
- => 「-」は1とし、「○-△階」は○階とする。また、地下はマイナス値とする。

In [15]:
# 「-」を「1階」に変換
df_floor = pd.DataFrame(df["階"], columns=["階"])
df_floor = pd.DataFrame(["1階" if x == "-" else x for x in df_floor["階"]], columns=["階"])
#print(Counter(df_floor1["階"]))
# 「○-△階」の処理
df_floor = df_floor["階"].str.split("-", expand=True)
df_floor.columns = ["階", "その他"]
# Bはマイナス値とし、階数を取得
df_floor = pd.DataFrame(df_floor["階"].str.replace("階", ""), columns=["階"])
df_floor = pd.DataFrame(df_floor["階"].str.replace("B", "-"), columns=["階"])
df_floor["階"] = pd.to_numeric(df_floor["階"])
# データ確認
Counter(df_floor["階"])

Counter({-1: 1,
         1: 3379,
         2: 4185,
         3: 2360,
         4: 841,
         5: 443,
         6: 249,
         7: 155,
         8: 127,
         9: 85,
         10: 47,
         11: 15,
         12: 16,
         13: 13,
         14: 11,
         15: 1,
         16: 1})

### 賃料

- => 全て「〇〇万円」表記なので、〇〇のみ取得

In [22]:
# 賃料の数値のみ取得し、10,000倍する。
df_rent = pd.DataFrame(round(pd.to_numeric(df["賃料"].str.replace("万円", "")) * 10000, 0), columns=["賃料"])
# 賃料の数値のみ取得し、10,000倍する。
df_rent = pd.DataFrame(round(pd.to_numeric(df["賃料"].str.replace("万円", "")) * 10000, 0), columns=["賃料"])
# データ確認
df_rent
Counter(df_rent["賃料"])

Counter({25000.0: 4,
         26000.0: 6,
         27000.0: 9,
         28000.0: 8,
         29000.0: 27,
         30000.0: 30,
         31000.0: 8,
         32000.0: 24,
         33000.0: 20,
         34000.0: 25,
         35000.0: 63,
         36000.0: 28,
         37000.0: 72,
         38000.0: 61,
         39000.0: 87,
         40000.0: 93,
         41000.0: 37,
         41100.0: 1,
         41500.0: 1,
         42000.0: 58,
         42500.0: 1,
         43000.0: 93,
         43500.0: 1,
         44000.0: 59,
         45000.0: 196,
         46000.0: 104,
         47000.0: 100,
         48000.0: 156,
         48500.0: 3,
         49000.0: 107,
         50000.0: 312,
         50500.0: 4,
         51000.0: 75,
         51500.0: 2,
         52000.0: 160,
         52300.0: 2,
         52500.0: 1,
         53000.0: 263,
         53500.0: 4,
         53700.0: 5,
         54000.0: 164,
         55000.0: 281,
         55500.0: 6,
         56000.0: 144,
         56500.0: 12,
         57000.0

### 管理費

- 「-」を0円とし、数値のみ取得

In [25]:
df_admin = pd.DataFrame(df["管理費"].str.replace("円", ""), columns=["管理費"])
df_admin = pd.DataFrame(pd.to_numeric(["0" if x == "-" else x for x in df_admin["管理費"]]) , columns=["管理費"])
# データ確認
Counter(df_admin["管理費"])

Counter({0: 2580,
         100: 2,
         150: 1,
         200: 1,
         300: 4,
         500: 6,
         1000: 288,
         1040: 1,
         1500: 19,
         1600: 1,
         1750: 1,
         2000: 1685,
         2100: 6,
         2200: 19,
         2250: 1,
         2300: 3,
         2400: 1,
         2425: 3,
         2475: 5,
         2500: 73,
         2501: 3,
         2576: 3,
         2635: 1,
         2700: 3,
         2900: 5,
         2905: 1,
         2935: 1,
         2975: 2,
         3000: 2886,
         3015: 10,
         3050: 2,
         3110: 1,
         3200: 5,
         3300: 3,
         3350: 1,
         3475: 1,
         3500: 277,
         3515: 3,
         3600: 4,
         3640: 1,
         3700: 2,
         3900: 10,
         3975: 1,
         4000: 637,
         4015: 3,
         4270: 11,
         4300: 1,
         4320: 12,
         4400: 5,
         4500: 115,
         4505: 1,
         4515: 2,
         4540: 1,
         4820: 3,
         500

### 敷/礼/保証/敷引,償却

In [26]:
Counter(df["敷/礼/保証/敷引,償却"])

Counter({'8.4万円/-/-/-': 20,
         '10.73万円/-/-/-': 1,
         '-/13.4万円/13.4万円/13.4万円': 2,
         '15.3万円/15.3万円/-/-': 1,
         '14.28万円/-/-/-': 3,
         '9万円/-/-/9万円': 4,
         '-/-/4.71万円/-': 2,
         '5万円/2.15万円/-/-': 1,
         '25万円/12.5万円/-/-': 1,
         '-/12万円/-/-': 2,
         '-/6.45万円/-/-': 17,
         '10.4万円/-/-/-': 3,
         '35.6万円/17.8万円/-/-': 1,
         '7.15万円/7.15万円/-/-': 12,
         '36.8万円/18.4万円/-/-': 2,
         '21.26万円/-/-/-': 1,
         '6.8万円/-/-/6.8万円': 5,
         '8.6万円/8.6万円/-/-': 20,
         '-/3.35万円/-/-': 6,
         '-/3.95万円/-/-': 1,
         '-/-/6.3万円/-': 4,
         '5.23万円/-/-/-': 2,
         '2.95万円/2.95万円/-/-': 2,
         '6.9万円/6.9万円/-/-': 55,
         '-/9.7万円/-/-': 4,
         '17.08万円/-/-/-': 1,
         '14.3万円/14.3万円/-/-': 3,
         '13.84万円/-/-/-': 1,
         '16.54万円/-/-/-': 2,
         '8.6万円/-/-/-': 12,
         '-/8.4万円/-/-': 19,
         '3.5万円/-/-/3.5万円': 6,
         '9.1万円/-/-/-': 2,
         '20.6万

- "/"で分割し、「-」は0円とし、数値を取得する。
- 万円単位なので10,000倍する。

In [29]:
# 「-」を0に変換
df_others = pd.DataFrame(df["敷/礼/保証/敷引,償却"].str.replace("-", "0"), columns=["敷/礼/保証/敷引,償却"])
# "万円"を削除
df_others = pd.DataFrame(df_others["敷/礼/保証/敷引,償却"].str.replace("万円", ""), columns=["敷/礼/保証/敷引,償却"])
# 数値取得し10,000倍
df_others = df_others["敷/礼/保証/敷引,償却"].str.split("/", expand=True).astype(float) * 10000
df_others.columns = ["敷金", "礼金", "保証金", "敷引,償却費"]
# データ確認
df_others

,敷金,礼金,保証金,"敷引,償却費"
0,25000.0,25000.0,0.0,0.0
1,26000.0,26000.0,0.0,0.0
2,25000.0,25000.0,0.0,0.0
3,25000.0,25000.0,0.0,0.0
4,25000.0,25000.0,0.0,0.0
5,28000.0,0.0,0.0,0.0
6,28000.0,0.0,0.0,0.0
7,28000.0,28000.0,0.0,0.0
8,28000.0,28000.0,0.0,0.0
9,28000.0,28000.0,0.0,0.0


### 間取り

- Suumoでの部屋検索より、「部屋数」「D有無」「K有無」「L有無」「S有無」に分割する。

In [42]:
# ここは敢えてarrでの変換で処理する。
df_floor_plan = df["間取り"].str.replace("ワンルーム", "1")
# arr作成
l = len(df_floor_plan)
arr_rooms = np.zeros(l)
arr_d = np.zeros(l)
arr_k = np.zeros(l)
arr_l = np.zeros(l)
arr_s = np.zeros(l)
# 格納していく
for i in range(0, l):
    plan = str(df_floor_plan[i])
    # D有無
    if "D" in plan:
        arr_d[i] = 1
    plan = plan.replace("D", "")
    # K有無
    if "K" in plan:
        arr_k[i] = 1
    # L有無
    if "L" in plan:
        arr_l[i] = 1
    # S有無
    if "S" in plan:
        arr_s[i] = 1
    for t in ["D","K","L","S"]:
        plan = plan.replace(t, "")
    arr_rooms[i] = plan

# データ確認
print(Counter(arr_rooms))
print(Counter(arr_d))
print(Counter(arr_k))
print(Counter(arr_l))
print(Counter(arr_s))

# DataFrameへ変換
df_floor_plan = pd.DataFrame(columns=["部屋数","D有無","K有無","L有無","S有無"])
df_floor_plan["部屋数"] = arr_rooms
df_floor_plan["D有無"] = arr_d
df_floor_plan["K有無"] = arr_k
df_floor_plan["L有無"] = arr_l
df_floor_plan["S有無"] = arr_s
# データ確認
df_floor_plan

Counter({1.0: 7673, 2.0: 3364, 3.0: 794, 4.0: 79, 5.0: 14, 6.0: 4, 7.0: 1})
Counter({0.0: 6970, 1.0: 4959})
Counter({1.0: 10353, 0.0: 1576})
Counter({0.0: 9593, 1.0: 2336})
Counter({0.0: 11761, 1.0: 168})


,部屋数,D有無,K有無,L有無,S有無
0,2.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0
9,1.0,0.0,0.0,0.0,0.0


### 専有面積

- => 全て"m"を削除し数値化

In [43]:
df_area = pd.DataFrame(round(pd.to_numeric(df["専有面積"].str.replace("m", "")), 2), columns=["専有面積"])
# データ確認
df_area

,専有面積
0,24.00
1,11.57
2,12.00
3,10.01
4,10.01
5,10.00
6,10.00
7,10.00
8,10.00
9,10.00


## 列の作成・結合など

- ここで、参考サイトより、求めたい「賃料」＝「管理費を含む月額費用」と定義すること、「敷/礼/保」＝「初期費用」と捉え、「月額費用」と「初期費用」を追加する。

In [58]:
# 月額費用・初期費用
df_cost = pd.DataFrame(columns=["月額費用", "初期費用"])
df_cost["月額費用"] = df_rent["賃料"] + df_admin["管理費"]
df_cost["初期費用"] = df_others["敷金"] + df_others["礼金"] + df_others["保証金"] + df_others["敷引,償却費"]
# データ確認
df_cost

,月額費用,初期費用
0,26000.0,50000.0
1,26000.0,52000.0
2,26500.0,50000.0
3,26500.0,50000.0
4,26500.0,50000.0
5,28000.0,28000.0
6,28000.0,28000.0
7,28000.0,56000.0
8,28000.0,56000.0
9,28000.0,56000.0


## データフレーム作成・csv保存

In [ ]:
df_all

# 関数作成

## ベースurlからurlリスト(urls)作成

In [3]:
def get_urls(base_url = "", url = ""):
    '''
    base_url: url検索を開始するベースurl
    urls: urlリストを格納するリスト
    '''
    urls = []
    
    if base_url == "":
        base_url = "http://suumo.jp"
    print("1\t", url)

    # 格納していく
    urls.append(url)
    flg = True
    i = 1
    while flg:
        #if i == 300:
        #    break
        i += 1
        request = requests.get(url)
        c = request.content
        soup = BeautifulSoup(c)
        # 次ページのリンクが入った<p>を取得
        next_page_ls = soup.find_all("p",{"class":"pagination-parts"})
        for next_page in next_page_ls:
            flg = True
            next_page_link = next_page.a
            # 次のページが見つからなかったら終了
            if "次へ" not in next_page_link:
                flg = False
            # 次のページが見つかれば、リンク取得
            if flg:
                url = base_url + next_page_link["href"]
                print(i, "\t", url)
                urls.append(url)
                # プログラムを最低1秒間停止する（スクレイピングマナー）
                time.sleep(4)
                
    print("終了：ページ数 = {}".format(str(i-1)))
    return urls

## 対象urlから物件情報を取得

In [4]:
def get_cassetteitems_info(urls):
    '''
    urls: 物件検索結果のurlリスト
    '''

    # データ（属性）を格納する変数を作成
    titles = [] #建物名（違う場合もあるが念のため）
    ltypes = [] #レンタルタイプ（賃貸アパート、賃貸マンション、など）
    addresses = [] #住所
    location0s = [] #立地1つ目（最寄駅/徒歩~分）
    location1s = [] #立地2つ目（最寄駅/徒歩~分）
    location2s = [] #立地3つ目（最寄駅/徒歩~分）
    ages = [] #築年数
    heights = [] #建物高さ
    floors = [] #階
    rents = [] #賃料
    admins = [] #管理費
    others = [] #敷/礼/保証/敷引,償却
    floor_plans = [] #間取り
    areas = [] #専有面積

    rooms = 0 # 物件数
    i = 0 # ページ数
    for url in urls:
        i += 1 #ページ数カウント
        # 物件情報が入っている箇所を取得
        result = requests.get(url)
        c = result.content
        soup = BeautifulSoup(c)
        summary = soup.find("div",{"id":"js-bukkenList"})

        # 物件（建物）ごとの情報を取得
        cassetteitems = summary.find_all("div",{"class":"cassetteitem"})

        rooms += len(cassetteitems) #物件数カウント
        print("{}/{}\t物件（建物）数：{}".format(str(i), str(len(urls)), len(cassetteitems)))
        # 各物件で情報を取得
        for cassetteitem in cassetteitems:
            # 部屋数を取得
            tbodies = cassetteitem.find_all("tbody")

            # 建物情報を取得：レンタルタイプ、建物名、住所、立地（3つまで）、築年数、建物高さ
            # - レンタルタイプ
            ltype = cassetteitem.find("span",{"class":"ui-pct ui-pct--util1"}).find(text=True)
            # - 建物名
            title = cassetteitem.find("div",{"class":"cassetteitem_content-title"}).find(text=True)
            # - 住所
            address = cassetteitem.find("li",{"class":"cassetteitem_detail-col1"}).find(text=True)
            # - 立地（3つまで）
            locations = cassetteitem.find_all("div",{"class":"cassetteitem_detail-text"})
            l = len(locations)
            if l == 1:
                location0 = str(locations[0].find(text=True))
            if l == 2:
                location0 = str(locations[0].find(text=True))
                location1 = str(locations[1].find(text=True))
            else:
                location0 = str(locations[0].find(text=True))
                location1 = str(locations[1].find(text=True))
                location2 = str(locations[2].find(text=True))
            # 築年数、建物高さ
            ah = cassetteitem.find("li",{"class":"cassetteitem_detail-col3"})
            ah = ah.find_all("div")
            age = str(ah[0].find(text=True))
            height = str(ah[1].find(text=True))

            # 部屋情報を取得：階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積
            for tbody in tbodies:
                cassetteitem_others = tbody.find_all("td")
                # 階
                floor = str(cassetteitem_others[2].find(text=True))
                # 賃料
                rent = str(cassetteitem_others[3].find("span",{"class":"cassetteitem_other-emphasis ui-text--bold"}).find(text=True))
                # 管理費
                admin = str(cassetteitem_others[4].find(text=True))
                # 敷/礼/保証/敷引,償却
                other = str(cassetteitem_others[5].find(text=True))
                # 間取り
                floor_plan = str(cassetteitem_others[6].find(text=True))
                # 専有面積
                area = str(cassetteitem_others[7].find(text=True))

                # 部屋ごとにデータ（属性）を格納
                titles.append(title) #建物名
                ltypes.append(ltype) #レンタルタイプ
                addresses.append(address) #住所
                location0s.append(location0) #立地1つ目
                location1s.append(location1) #立地2つ目
                location2s.append(location2) #立地3つ目
                ages.append(age) #築年数
                heights.append(height) #建物高さ
                floors.append(floor) #階
                rents.append(rent) #賃料
                admins.append(admin) #管理費
                others.append(other) #敷/礼/保証/敷引,償却
                floor_plans.append(floor_plan) #間取り
                areas.append(area) #専有面積            

        # プログラムを最低1秒間停止する（スクレイピングマナー）
        time.sleep(4)

    print("ページ数：{}\t物件数：{}\t部屋数：{}".format(str(i), str(rooms), str(len(ltypes))))

    # データ格納した変数をSeries化
    titles = pd.Series(titles) #建物名
    ltypes = pd.Series(ltypes) #レンタルタイプ
    addresses = pd.Series(addresses) #住所
    location0s = pd.Series(location0s) #立地1つ目
    location1s = pd.Series(location1s) #立地2つ目
    location2s = pd.Series(location2s) #立地3つ目
    ages = pd.Series(ages) #築年数
    heights = pd.Series(heights) #建物高さ
    floors = pd.Series(floors) #階
    rents = pd.Series(rents) #賃料
    admins = pd.Series(admins) #管理費
    others = pd.Series(others) #敷/礼/保証/敷引,償却
    floor_plans = pd.Series(floor_plans) #間取り
    areas = pd.Series(areas) #専有面積

    # 各SeriesをDataFrame化
    df = pd.concat([titles, ltypes, addresses, location0s, location1s, location2s, ages, heights, floors, rents, admins, others, floor_plans, areas], axis = 1)

    # 列名を追加
    df.columns = ["建物名", "レンタルタイプ", "住所", "立地1", "立地2", "立地3", "築年数", "建物高さ", "階", "賃料", "管理費", "敷/礼/保証/敷引,償却", "間取り", "専有面積"]

    return df

# 23区分のデータを一括取得

## 23区の1ページ目のurlリスト作成

In [5]:
all_first_urls = ["http://suumo.jp/chintai/tokyo/sc_chiyoda/",
                  "http://suumo.jp/chintai/tokyo/sc_chuo/",
                  "http://suumo.jp/chintai/tokyo/sc_minato/",
                  "http://suumo.jp/chintai/tokyo/sc_shinjuku/",
                  "http://suumo.jp/chintai/tokyo/sc_bunkyo/",
                  "http://suumo.jp/chintai/tokyo/sc_taito/",
                  "http://suumo.jp/chintai/tokyo/sc_sumida/",
                  "http://suumo.jp/chintai/tokyo/sc_koto/",
                  "http://suumo.jp/chintai/tokyo/sc_shinagawa/",
                  "http://suumo.jp/chintai/tokyo/sc_meguro/",
                  "http://suumo.jp/chintai/tokyo/sc_ota/",
                  "http://suumo.jp/chintai/tokyo/sc_setagaya/",
                  "http://suumo.jp/chintai/tokyo/sc_shibuya/",
                  "http://suumo.jp/chintai/tokyo/sc_nakano/",
                  "http://suumo.jp/chintai/tokyo/sc_suginami/",
                  "http://suumo.jp/chintai/tokyo/sc_toshima/",
                  "http://suumo.jp/chintai/tokyo/sc_kita/",
                  "http://suumo.jp/chintai/tokyo/sc_arakawa/",
                  "http://suumo.jp/chintai/tokyo/sc_itabashi/",
                  "http://suumo.jp/chintai/tokyo/sc_nerima/",
                  "http://suumo.jp/chintai/tokyo/sc_adachi/",
                  "http://suumo.jp/chintai/tokyo/sc_katsushika/",
                  "http://suumo.jp/chintai/tokyo/sc_edogawa/"]

In [6]:
ku_list = ["chiyoda", "chuo", "minato", "shinjuku", "bunkyo", "taito", "sumida", "koto", "shinagawa",
          "meguro", "ota", "setagaya", "shibuya", "nakano", "suginami", "toshima", "kita", "arakawa",
          "itabashi", "nerima", "adachi", "katsushika", "edogawa"]

## 23区のurlリストとdfを一括作成

In [7]:
for i in range(11,23):
    print("i={}\tku={}\turl={}".format(str(i),ku_list[i], all_first_urls[i]))

i=11	ku=setagaya	url=http://suumo.jp/chintai/tokyo/sc_setagaya/
i=12	ku=shibuya	url=http://suumo.jp/chintai/tokyo/sc_shibuya/
i=13	ku=nakano	url=http://suumo.jp/chintai/tokyo/sc_nakano/
i=14	ku=suginami	url=http://suumo.jp/chintai/tokyo/sc_suginami/
i=15	ku=toshima	url=http://suumo.jp/chintai/tokyo/sc_toshima/
i=16	ku=kita	url=http://suumo.jp/chintai/tokyo/sc_kita/
i=17	ku=arakawa	url=http://suumo.jp/chintai/tokyo/sc_arakawa/
i=18	ku= itabashi	url=http://suumo.jp/chintai/tokyo/sc_itabashi/
i=19	ku=nerima	url=http://suumo.jp/chintai/tokyo/sc_nerima/
i=20	ku=adachi	url=http://suumo.jp/chintai/tokyo/sc_adachi/
i=21	ku=katsushika	url=http://suumo.jp/chintai/tokyo/sc_katsushika/
i=22	ku=edogawa	url=http://suumo.jp/chintai/tokyo/sc_edogawa/


In [8]:
today = "20171119"
base_url = "http://suumo.jp"
#j = 0
#for first_url in all_first_urls:
for j in range(11, 23):
    first_url = all_first_urls[j]
    print("*"*100)
    print("{}区/23区\t{}".format(str(j+1), str(ku_list[j])))
    print("*"*100)
    # urlリストを作成
    urls = get_urls(base_url, first_url)
    # urlリストをcsvファイル出力
    urls_df = pd.DataFrame({"url":urls})
    urls_filename = "../output/urls/urls_" + ku_list[j] + "_" + today + ".csv"
    urls_df.to_csv(urls_filename, index=False, header=False)

    # 物件データを取得
    df = get_cassetteitems_info(urls)
    # 物件データをcsvファイル出力
    df_filename = "../output/info/info_" + ku_list[j] + "_" + today + ".csv"
    df.to_csv(df_filename, sep = ",", encoding = "utf-8")    
    
    # サイクルを回す
    #j += 1
    

****************************************************************************************************
12区/23区	setagaya
****************************************************************************************************
1	 http://suumo.jp/chintai/tokyo/sc_setagaya/


/Users/kimiaki/Documents/virtualenv/py3.5/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


2 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz12.html
3 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz13.html
4 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz14.html
5 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz15.html
6 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz16.html
7 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz17.html
8 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz18.html
9 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz19.html
10 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz110.html
11 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz111.html
12 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz112.html
13 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz113.html
14 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz114.html
15 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz115.html
16 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz116.html
17 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz117.html
18 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz118.html
19 	 http://s

140 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1140.html
141 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1141.html
142 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1142.html
143 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1143.html
144 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1144.html
145 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1145.html
146 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1146.html
147 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1147.html
148 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1148.html
149 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1149.html
150 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1150.html
151 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1151.html
152 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1152.html
153 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1153.html
154 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1154.html
155 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1155.html
156 	 http://suumo.jp/ch

275 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1275.html
276 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1276.html
277 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1277.html
278 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1278.html
279 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1279.html
280 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1280.html
281 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1281.html
282 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1282.html
283 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1283.html
284 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1284.html
285 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1285.html
286 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1286.html
287 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1287.html
288 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1288.html
289 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1289.html
290 	 http://suumo.jp/chintai/tokyo/sc_setagaya/pnz1290.html
291 	 http://suumo.jp/ch

...

137 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1137.html
138 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1138.html
139 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1139.html
140 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1140.html
141 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1141.html
142 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1142.html
143 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1143.html
144 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1144.html
145 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1145.html
146 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1146.html
147 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1147.html
148 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1148.html
149 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1149.html
150 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1150.html
151 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1151.html
152 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1152.html
153 	 http://suumo.jp/chintai/tokyo/sc_adachi/pnz1153.ht

180/219	物件（建物）数：30
181/219	物件（建物）数：30
182/219	物件（建物）数：30
183/219	物件（建物）数：30
184/219	物件（建物）数：30
185/219	物件（建物）数：30
186/219	物件（建物）数：30
187/219	物件（建物）数：30
188/219	物件（建物）数：30
189/219	物件（建物）数：30
190/219	物件（建物）数：30
191/219	物件（建物）数：30
192/219	物件（建物）数：30
193/219	物件（建物）数：30
194/219	物件（建物）数：30
195/219	物件（建物）数：30
196/219	物件（建物）数：30
197/219	物件（建物）数：30
198/219	物件（建物）数：30
199/219	物件（建物）数：30
200/219	物件（建物）数：30
201/219	物件（建物）数：30
202/219	物件（建物）数：30
203/219	物件（建物）数：30
204/219	物件（建物）数：30
205/219	物件（建物）数：30
206/219	物件（建物）数：30
207/219	物件（建物）数：30
208/219	物件（建物）数：30
209/219	物件（建物）数：30
210/219	物件（建物）数：30
211/219	物件（建物）数：30
212/219	物件（建物）数：30
213/219	物件（建物）数：30
214/219	物件（建物）数：30
215/219	物件（建物）数：30
216/219	物件（建物）数：30
217/219	物件（建物）数：30
218/219	物件（建物）数：30
219/219	物件（建物）数：11
ページ数：219	物件数：6551	部屋数：11929
****************************************************************************************************
22区/23区	katsushika
********************************************************************************************

119 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1119.html
120 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1120.html
121 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1121.html
122 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1122.html
123 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1123.html
124 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1124.html
125 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1125.html
126 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1126.html
127 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1127.html
128 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1128.html
129 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1129.html
130 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1130.html
131 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1131.html
132 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1132.html
133 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz1133.html
134 	 http://suumo.jp/chintai/tokyo/sc_katsushika/pnz11

69 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz169.html
70 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz170.html
71 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz171.html
72 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz172.html
73 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz173.html
74 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz174.html
75 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz175.html
76 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz176.html
77 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz177.html
78 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz178.html
79 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz179.html
80 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz180.html
81 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz181.html
82 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz182.html
83 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz183.html
84 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz184.html
85 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz185.html
86 	 http://su

207 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1207.html
208 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1208.html
209 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1209.html
210 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1210.html
211 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1211.html
212 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1212.html
213 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1213.html
214 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1214.html
215 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1215.html
216 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1216.html
217 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1217.html
218 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1218.html
219 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1219.html
220 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1220.html
221 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1221.html
222 	 http://suumo.jp/chintai/tokyo/sc_edogawa/pnz1222.html
223 	 http://suumo.jp/chintai/tokyo/sc_e

- => 終わってから、「ページアクセス→情報取得→次ページあればサイクル回す」にすれば時間半分で済んだなーと反省。
- 参考サイトの流れを意識しすぎたことが原因で、自分のコードに落とした場合を考慮し、修正が必要でした。ちゃんちゃん。

# データ前処理

# 予測モデル作成

# お買い得物件を探す